# Functional Encryption - Classification and information leakage

Our start point is the work on encrypted classification using Function Encryption of the paper [Reading in the Dark: Classifying Encrypted Digits with Functional Encryption](https://eprint.iacr.org/2018/206), and the associated [GitHub repository](https://github.com/edufoursans/reading-in-the-dark).

More specifically, the paper provides a new Functional Encryption scheme for quadratic multi-variate polynomials, which can under some hypothesis be seen as a single hidden layer neural network with a quadratic activation.
In the paper, the output corresponds to 1 element per class, so 10 on their example using MNIST, and it is made in clear. The argmax is then taken in clear to determine the class. This approach is standard and was used in [CryptoNets](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/04/CryptonetsTechReport.pdf), [MiniONN](https://eprint.iacr.org/2017/452.pdf), [CryptoDL](https://arxiv.org/pdf/1711.05189.pdf) and other recent papers on Homomorphic Encryption (HE) (like [this paper of 2018](https://arxiv.org/pdf/1807.08459.pdf)).


However, there has been very little discussion about the sensitivy of this output, because in the case of MNIST 10 real values are output, which in term of bits of information is huge. Our starting point is therefore: can this output disclose extra information about the initial input or about charasteristics of this input, than just the label we want to classify.

To this aim, we have built a specific dataset which is very similar to MNIST, used in the original paper but which is composed of 10 letter characters of 2 differents fonts. Our goal is two-fold:

 - Evaluate how the output in clear can be leveraged with a public neural network to make better predictions than a simple `argmax` function on the character recognition task.
 - Analyse to what extent the output in clear of the model can reveal information about the font used, using an "collateral" network.
 
But first, let's build this dataset!

 > Writer identification is not a novel task, several datasets have proposed among which the CDAR 2013 Competition  on  Writer  Identification, ICDAR  2011  Writer  Identification  Contest, and the CVL-Database datasets. However these datasets don't consider characters but complete pages and text extracts which is not suitable for our analysis.
 
 > EMNIST however could be an interesting dataset but they don't provide the two labels (value, authors) together. This dataset will be used later to confirm results.
 
 > There is also this excellent work @TalwalkarLab to create [FEMNIST](https://github.com/TalwalkarLab/leaf) which can be used.

# 1. Build the dataset

But first, let's build the dataset!

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from string import ascii_lowercase
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter
import torch.utils.data as utils
import pickle

## 1.1 Fonts

We have done previous work on font distinguishability in a classification task and we've select the two fonts *cursive, Georgia* that are of medium difficulty to distinguish. We will extend our analysis two more couples but this is a fair start.

In [ ]:
families = ['cursive', 'Georgia']

Here what they look like!

In [ ]:
plt.figure(figsize=(12,2))
for i, family in enumerate(families):
    ax = plt.subplot(1, 5, i+1)
    ax.set_title(family)
    ax.text(0.3, 0.4, '123', size=50, family=family)

    # Rm axes, draw and get the rgba shape of the digits
    plt.axis('off')
plt.show()

## 1.2 Noising process
Our dataset is artificial. Hence we need to add some noise to make it more interesting for a classification task.
Currently the noise boils down to:

 - A random but moderate rotation
 - A random but moderation size variation
 - A deformation using a Gaussian filter

When we are done, we recenter the data (as it is done for MNIST).

Let's do it for one character to see how it looks like!

### Utils functions to handle rgb / rgba conversion

In [ ]:
def rgba_to_rgb(rgba):
        if rgba[0] > 0:
            return np.ones(3) * (256 - rgba[0])
        return rgba[1:]
    
def convert_to_rgb(data):
    return np.apply_along_axis(rgba_to_rgb, 2, data) 

### Deformation function

In [ ]:
def elastic_transform(image, alpha, sigma, random_state=None):
    """Elastic deformation of images as described in [Simard2003]_.
    .. [Simard2003] Simard, Steinkraus and Platt, "Best Practices for
       Convolutional Neural Networks applied to Visual Document Analysis", in
       Proc. of the International Conference on Document Analysis and
       Recognition, 2003.
    """
    if random_state is None:
        random_state = np.random.RandomState(None)

    shape = image.shape
    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
    dz = np.zeros_like(dx)

    x, y, z = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]), np.arange(shape[2]))
    indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1)), np.reshape(z, (-1, 1))

    distored_image = map_coordinates(image, indices, order=1, mode='reflect')
    return distored_image.reshape(image.shape)

### Recentering function

In [ ]:
def center(data):
    # Inverse black and white
    wb_data = np.ones(data.shape) * 255 - data
    
    # normalize
    prob_data = wb_data / np.sum(wb_data)
    
    # marginal distributions
    dx = np.sum(prob_data, (1, 2))
    dy = np.sum(prob_data, (0, 2))

    # expected values
    (X, Y, Z) = prob_data.shape
    cx = np.sum(dx * np.arange(X))
    cy = np.sum(dy * np.arange(Y))
    
    # Check bounds
    assert cx > X/4 and cx < 3 * X/4, f"ERROR: {cx} > {X/4} and {cx} < {3 * X/4}"
    assert cy > Y/4 and cy < 3 * Y/4, f"ERROR: {cy} > {Y/4} and {cy} < {3 * Y/4}"
    
    # print('Center', cx, cy)
    
    x_min = int(round(cx - X/4))
    x_max = int(round(cx + X/4))
    y_min = int(round(cy - Y/4))
    y_max = int(round(cy + Y/4))
    
    return data[x_min:x_max, y_min:y_max, :]
    

### One first example!
Ok let's how it looks like!

In [ ]:
# Sample a rotation angle
rotation = np.random.normal(0, 10) # 10 degrees (out of 180)
# Sample a text size +/- 5% in std
size = 100 + np.random.normal(0, 5) 
# Sample a family
family_idx = np.random.randint(len(families))
family = families[family_idx]
# Sample a char
digit = np.random.randint(10)
letter = str(digit)

# Show rotated letter
fig = plt.figure(figsize=(2,2), dpi=100)
fig.text(0.4, 0.4, letter, size=size, rotation=rotation, family=family)
plt.axis('off')
plt.show()
plt.close(fig)

# Load the letter in smaller size
fig = plt.figure(figsize=(2,2), dpi=100)
fig.text(0.4, 0.4, letter, size=50, rotation=rotation, family=family)

# Rm axes, draw and get the rgba shape of the letter
plt.axis('off')
fig.canvas.draw()
data = np.frombuffer(fig.canvas.tostring_argb(), dtype=np.uint8)
data = data.reshape(fig.canvas.get_width_height()[::-1] + (4,))
plt.close(fig)
# Convert to rgb
data = convert_to_rgb(data)

# Re-center the data
data = center(data)

plt.show()
plt.axis('off')
plt.imshow(data)
plt.show()

# Apply an elastic deformation
data = elastic_transform(data, alpha=991, sigma=9)

plt.axis('off')
plt.imshow(data)
plt.show()

# Free memory space
plt.close(fig)

Notice that the images here are provided with high resolution, they will then be converted to 28x28 pixels and will becomes way harder to analyse.

Feel free to change the rotation std, the text size std, and the `alpha` and `sigma` parameters in the deformation. You'll see that the task can be made incredibly harder!

### Building the dataset
Ok now we are equipped to build a complete dataset!

In [ ]:
# We define a variance parameter for rotation and one for text size
std_rotation = 10
std_size = 2.5 # 5% of reference size, here 50

In [ ]:
%%time
def build_dataset(dataset_size, verbose=False):
    dataset_data = []
    dataset_target_char = []
    dataset_target_family = []
    for i in range(dataset_size):
        if i % int(dataset_size/100) == 0:
            print(round(i / dataset_size * 100), '%')
        rotation = np.random.normal(0, std_rotation)
        size = 50 + np.random.normal(0, std_size) 
        family_idx = np.random.randint(len(families))
        family = families[family_idx]
        digit = np.random.randint(10)
        letter = str(digit)

        fig = plt.figure(figsize=(2,2), dpi=28)
        fig.text(0.4, 0.4, letter, size=size, rotation=rotation, family=family)

        # Rm axes, draw and get the rgba shape of the letter
        plt.axis('off')
        fig.canvas.draw()
        data = np.frombuffer(fig.canvas.tostring_argb(), dtype=np.uint8)
        data = data.reshape(fig.canvas.get_width_height()[::-1] + (4,))

        # Convert to rgb
        data = convert_to_rgb(data)

        # Center the data
        data = center(data)

        if verbose:
            plt.show()
            plt.axis('off')
            plt.imshow(data)
            plt.show()
            
        # Apply an elastic deformation
        data = elastic_transform(data, alpha=991, sigma=9)

        if verbose:
            plt.axis('off')
            plt.imshow(data)
            plt.show()

        # Free memory space
        plt.close(fig)

        # Append data to the datasets
        target_char = np.array([digit])
        target_family = np.array([family_idx])
        dataset_data.append(data[:,:,0])
        dataset_target_char.append(target_char)
        dataset_target_family.append(target_family)
        
    return dataset_data, dataset_target_char, dataset_target_family


def save_dataset(info, data, target_char, target_family):
    with open('dataset/character_dataset_{}.pkl'.format(info), 'wb') as output:
        dataset = data, target_char, target_family
        pickle.dump(dataset, output, pickle.HIGHEST_PROTOCOL)

# We build 10000 per 10000 for memory purposes
# WARNING: This step takes time, and you may need to restart you notebook for memory purposes.
for i in range(6):
    print(f"Train{i}")
    train_data, train_target_char, train_target_family = build_dataset(10000)
    save_dataset(f"train{i}", train_data, train_target_char, train_target_family)
    
print("Test")
test_data, test_target_char, test_target_family = build_dataset(10000)
save_dataset("test", test_data, test_target_char, test_target_family)

Ok we now have our dataset! Let's use it!

### Appendix
Here is an utility function to draw some samples of the dataset

In [ ]:
# Allow to load packages from parent
import sys, os
sys.path.insert(1, os.path.realpath(os.path.pardir))
import random, math
import learn
data = learn.load_data()
train_data, train_target_char, train_target_family, test_data, test_target_char, test_target_family = data

In [ ]:
def show_examples(data, target_char, target_family, n_examples=20):
    n_rows = math.ceil(n_examples / 5)
    plt.figure(figsize=(14, 3*n_rows))
    for i in range(n_examples):
        ax = plt.subplot(n_rows, 5, i+1)
        idx = random.randint(0, len(data))
        image = data[idx]
        letter = str([target_char[idx][0]])
        family = families[target_family[idx][0]]
        ax.set_title(f"{letter} ({family})")
        plt.axis('off')
        ax.imshow(image, cmap='gist_gray')
    plt.show()
        
        
show_examples(test_data, test_target_char, test_target_family)

It now much harder to distinguish the fonts, isn't it?!

Ok let's use this dataset to start out work!


### If you like it, star it!

The easiest way to show support is just by starring the Repos! This helps raise awareness on this topics and is a precious feedback for the repo maintainers!

- [Star the Repo](https://github.com/LaRiffle/collateral-learning)